# Thông tin sinh viên

**Họ tên**: Dương Trường Bình

**MSSV**: 21127229

# Giải thuật chéo hóa ma trận

1. Giải $det(A-\lambda I)=0$ để tìm các trị riêng
2. Với trị riêng $\lambda_i$ Xét $E_{\lambda_i}=\{v|(A-\lambda_iI)v=0\}$  để tìm vector riêng $v_i$ tương ứng với $\lambda_i$   
3. Nếu số vector riêng tìm được bằng số chiều của không gian thì ma trận $A$ chéo hóa được
	- Ma trận P có các cột là các vector riêng tìm được
	- Tính $D=P^{-1}AP$ chính là ma trận chéo hóa của $A$
	- Trả về $P$, $P^{-1}$, $D$


In [1]:
def print_matrix(matrix):
    m = len(matrix)
    n = len(matrix[0])
    
    print('[', end='')
    for i in range(m):
        if i != 0:
            print(' ', end='')
        print('[', end='')
        for j in range(n):
            print(f'{matrix[i][j]:5.2f}', end=' ')
        print(']', end='')
        if i < m - 1:
            print()
    print(']')
    print()


In [2]:
def dot_product(v1, v2):
    if len(v1) != len(v2):
        raise Exception('Vector length mismatch')
    return sum([i1 * i2 for i1, i2 in zip(v1, v2)])


def transpose(matrix):
    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]


def dot_matrix(m1, m2):
    if len(m1[0]) != len(m2):
        raise Exception('Matrix dimension mismatch')
    return [[dot_product(m1[i], [m2[j][k] for j in range(len(m2))]) for k in range(len(m2[0]))] for i in range(len(m1))]

In [3]:
def determinant(A):
    # Tính định thức của ma trận A - lambda*I
    # Trả về một đa thức
    size = len(A)
    det = 0
    if size == 1:
        return A[0][0]
    elif size == 2:
        return (A[0][0])*(A[1][1]) - A[0][1]*A[1][0]
    elif size >= 3:
        for i in range(size):
            det += ((-1)**i)*A[0][i]*determinant(minor(A, 0, i))
        return det


def minor(A, i, j):
    # Phần bù đại số của ma trận A tại vị trí (i,j)
    return [row[:j] + row[j+1:] for row in (A[:i]+A[i+1:])]

In [4]:
def inverse(matrix):
    # Kích thước của ma trận vuông
    size = len(matrix)
    
    # Ghép thêm ma trận đơn vị vào bên phải
    matrix = [matrix[i] + [1 if i == j else 0 for j in range(size)] for i in range(size)]
    
    # Biến đổi Gauss-Jordan để đưa về ma trận đường chéo, lặp qua từng cột
    for index in range(size):
        # Nếu phần tử pivot bằng 0 thì tìm dòng đầu tiên khác 0 để hoán vị hai dòng
        if matrix[index][index] == 0:
            for i in range(index + 1, size):
                if matrix[i][index] != 0:
                    matrix[index], matrix[i] = matrix[i], matrix[index]
                    break
            # Nếu không tìm được thì tức là cả cột từ vị trí index đến n đều bằng 0 => ma trận không khả nghịch
            else:
                print('Ma trận không khả nghịch')
                return

        # Chia dòng đang xét cho phần tử pivot để đưa về 1    
        factor = matrix[index][index]
        matrix[index] = [i / factor for i in matrix[index]]
        
        # Biến đổi các số hạng trên cột index của các dòng khác thành 0
        for i in range(size):
            if i != index:
                matrix[i] = [matrix[i][j] - matrix[i][index] * matrix[index][j] for j in range(size * 2)]
        
        
    # Ma trận nghịch đảo là ma trận các cột bên phải của ma trận sau khi biến đổi
    result = [row[size:] for row in matrix]
            
    return result

In [5]:
def Gauss_elimination(matrix):
    
    m = len(matrix)  # Số hàng của ma trận
    n = len(matrix[0])  # Số cột của ma trận
    
    # Tọa độ của pivot
    pivot_i = 0
    pivot_j = 0
    
    while pivot_i <= m - 1 and pivot_j <= n - 2:
        # Kiểm tra pivot nếu bằng 0 thì tìm dòng khác 0 và đổi chỗ
        if matrix[pivot_i][pivot_j] == 0:
            for k in range(pivot_i + 1, m):
                if matrix[k][pivot_j] != 0:
                    matrix[pivot_i], matrix[k] = matrix[k], matrix[pivot_i]
                    break
            # Nếu cả cột đều bằng 0 thì chuyển sang cột kế tiếp
            else:
                pivot_j += 1
                continue
                
        # Nhân dòng chứa pivot với 1/pivot để có số dẫn đầu là 1
        factor = matrix[pivot_i][pivot_j]
        matrix[pivot_i] = [element / factor for element in matrix[pivot_i]]

        # Cộng một bội số thích hợp của dòng đó cho từng dòng dưới để biến các số hạng bên dưới số dẫn đầu thành 0.
        # Vì dòng có số dẫn đầu là 1 nên bội số là chính các số hạng của các dòng dưới
        for k in range(pivot_i + 1, m):
            matrix[k] = [matrix[k][x] - matrix[k][pivot_j] * matrix[pivot_i][x]
                         for x in range(n)]
            
        # Tiếp tục đến khi đạt được ma trận bậc thang
        pivot_i += 1
        pivot_j += 1

    return matrix


In [6]:
def back_substitution(matrix):

    m = len(matrix)  # Số hàng của ma trận
    n = len(matrix[0])  # Số cột của ma trận

    # Danh sách chứa các nghiệm của hệ phương trình
    solutions = [0] * (n - 1)

    # Số lượng dòng có toàn số 0
    zero_row = 0

    for row in matrix[::-1]:
        # Trường hợp vô nghiệm

        # Dòng có hệ số toàn 0 nhưng hệ số tự do khác 0
        if all(i == 0 for i in row[:-1]) and row[-1] != 0:
            print('Hệ phương trình vô nghiệm')
            return
        # Đếm số dòng toàn 0
        elif all(i == 0 for i in row):
            zero_row += 1

    # Số dòng khác 0
    non_zero_row = m-zero_row
    # Số nghiệm tùy ý
    count = n - 1 - non_zero_row
    # Biến chỉ mục để lặp từ dòng dưới cùng lên trên
    i = non_zero_row - 1
    # Danh sách chứa các ẩn và biến tùy ý của ẩn nếu có
    variables = [[0] * (n - 1) for _ in range(count)]

    
    if non_zero_row == n - 1:
        # Trường hợp nghiệm duy nhất
        while i >= 0:
            temp = matrix[i][n - 1]
            for j in range(n - 2, i, -1):
                temp -= matrix[i][j] * solutions[j]
            solutions[i] = round(temp, 5)
            i -= 1
            

    elif non_zero_row < n - 1:
        # Trường hợp vô số nghiệm
        
        # List để duyệt xem ẩn nào có biến tùy ý
        temp_list = [1] * (n - 1)
        # Tìm pivot từng dòng và gán vào temp_list, phần tử nào của list = 1 sẽ có biến tùy ý
        for row in range(non_zero_row):
            pivot_index = matrix[row].index(1)
            temp_list[pivot_index] = 0

        # Gán vào variables để biết ẩn nào có biến tùy ý (nếu là 1 thì ẩn đó có biến tùy ý)
        v = 0
        for index in range(len(temp_list)):
            if temp_list[index] == 1:
                variables[v][index] = 1
                v += 1

        # Thế ngược và tìm nghiệm
        while i >= 0:
            # Hệ số tự do của phương trình (hệ số sau dấu =)
            temp = matrix[i][n - 1]
            j = n - 2
            pivot_index = matrix[i].index(1)

            # Lặp đến pivot_index thì dừng
            while j > pivot_index:
                # Dùng phép trừ vì chuyển vế đổi dấu
                temp -= matrix[i][j] * solutions[j]
                # Tính các hệ số của các biến tùy ý trong variables
                for x in range(len(variables)):
                    variables[x][pivot_index] -= round(
                        matrix[i][j] * variables[x][j], 5)
                j -= 1
            # Gán nghiệm vào list chứa nghiệm với pivot_index tương ứng
            solutions[pivot_index] = round(temp, 5)
            i -= 1

    return variables

In [7]:
import sympy as sp


def diagonalize(A):
    # Tìm eigenvalues và eigenvectors
    A_minus_lambda_I = [[A[i][j] - sp.Symbol('lambda') if i == j else A[i][j]
                         for j in range(len(A[0]))] for i in range(len(A))]
    # Giải phương trình det(A - lambda*I) = 0
    det_A_minus_lambda_I = determinant(A_minus_lambda_I)
    eigenvalues = sp.solve(det_A_minus_lambda_I, sp.Symbol('lambda'))
    eigenvalues = [round(eigenvalue.evalf(), 5) for eigenvalue in eigenvalues]

    eigenvectors = []
    for eigenvalue in eigenvalues:

        matrix = [[A[i][j] - eigenvalue if i == j else A[i][j]
                   for j in range(len(A[0]))] for i in range(len(A))]

        # Thêm cột 0 vào cuối ma trận
        matrix = [row + [0] for row in matrix]

        matrix = Gauss_elimination(matrix)
        
        for eigenvector in back_substitution(matrix):
            if eigenvector not in eigenvectors:
                eigenvectors.append(eigenvector)

    if len(eigenvectors) != len(A):
        print(' Ma trận A không chéo hóa được')
        return (None, None, None)

    # Ma trận P chính là ma trận các eigenvectors và ma trận D là ma trận chéo hóa
    P = eigenvectors
    P = transpose(P)

    P_inverse = inverse(P)
    # D = P^-1 * A * P
    D = dot_matrix(dot_matrix(P_inverse, A), P)

    return P, P_inverse, D

In [8]:
A = [
    [4, 0, 1],
    [-2, 1, 0],
    [-2, 0, 1]
]
C = [
    [-1, 3],
    [-2, 4]
]

E = [[2, 1, 0],
     [0, 3, 1],
     [0, 0, 4]]
F = [
    [5,-1,1],
    [-1,2,-2],
    [1,-2,2]
]
G= [[-7,0,5],
              [0, 3, 0],
              [-10, 0, 8]]
P, P_inverse, D = diagonalize(G)
if P and P_inverse and D:
    print('P = ')
    print_matrix(P)
    print('P_inverse = ')
    print_matrix(P_inverse)
    print('D = ')
    print_matrix(D)



[[ 0.00  1.00  0.00  0.00 ]
 [ 0.00  0.00  1.00  0.00 ]
 [ 0.00  0.00  0.00  0.00 ]]

[[ 1.00 -1.00  0.00  0.00 ]
 [ 0.00  0.00  1.00  0.00 ]
 [ 0.00  0.00  0.00  0.00 ]]

[[ 1.00 -0.50  0.00  0.00 ]
 [ 0.00  1.00 -1.00  0.00 ]
 [ 0.00  0.00  0.00  0.00 ]]

P = 
[[ 1.00  1.00  0.50 ]
 [ 0.00  1.00  1.00 ]
 [ 0.00  0.00  1.00 ]]

P_inverse = 
[[ 1.00 -1.00  0.50 ]
 [ 0.00  1.00 -1.00 ]
 [ 0.00  0.00  1.00 ]]

D = 
[[ 2.00  0.00  0.00 ]
 [ 0.00  3.00  0.00 ]
 [ 0.00  0.00  4.00 ]]



## Ý tưởng thực hiện



# Mô tả các hàm

## print_matrix

- **Input**:
    - matrix: Ma trận đầu vào
- **Output**: None
- **Mục đích**: In ra ma trận được căn chỉnh cho dễ nhìn
- **Hoạt động**: Lặp qua từng phần tử và in ra nó


## dot_product
- **Input**: Hai vector v1 và v2.
- **Output**: Tích vô hướng của hai vector.
- **Mục đích**: Hàm tính tích vô hướng của hai vector v1 và v2.
- **Hoạt động**: 
    - Kiểm tra nếu độ dài của v1 khác độ dài của v2, nếu có thì gây ra một ngoại lệ với thông báo "Vector length mismatch".
    - Sử dụng list comprehension và hàm zip để tính tích vô hướng của hai vector
    - Trả về kết quả tích vô hướng.

## transpose
- **Input**: Một ma trận matrix.
- **Output**: Ma trận chuyển vị của matrix.
- **Mục đích**: Hàm tính ma trận chuyển vị của matrix.
- **Hoạt động**: 
    - Sử dụng list comprehension và hai vòng lặp để tạo ma trận chuyển vị.
    - Trong vòng lặp bên ngoài, duyệt qua từng cột của matrix (theo chiều dọc).
    - Trong vòng lặp bên trong, duyệt qua từng hàng của matrix (theo chiều ngang).
    - Trong cùng một vòng lặp bên trong, trích xuất từng phần tử từ các hàng tương ứng của matrix và sắp xếp chúng thành một cột.
    - Trả về kết quả ma trận chuyển vị.

## dot_matrix
- **Input**: Hai ma trận m1 và m2.
- **Output**: Một ma trận, là tích của m1 với m2, theo quy tắc nhân ma trận.
- **Mục đích**: Hàm tính tích của hai ma trận m1 và m2.
- **Hoạt động**: 
    - Sử dụng list comprehension và hai vòng lặp để tính tích của hai ma trận theo quy tắc nhân ma trận.
    - Trong vòng lặp bên ngoài, duyệt qua từng hàng của m1.
    - Trong vòng lặp bên trong, duyệt qua từng cột của m2.
    - Sử dụng hàm dot_product để tính tích vô hướng của hàng hiện tại của m1 với cột hiện tại của m2.
    - Trả về kết quả tích.


## minor
- **Input**:
    - A: Ma trận đầu vào
    - i: chỉ số hàng cần loại bỏ
    - j: chỉ số cột cần loại bỏ
- **Output**: Ma trận con của ma trận đầu vào sau khi loại bỏ hàng i và cột j
- **Mục đích**: Hàm trả về phần bù đại số của ma trận đầu vào sau khi loại bỏ hàng i và cột j
- **Hoạt động**:
    - Dùng list comprehension để cắt bỏ hàng i và cột j của ma trận đầu vào


## determinant
- **Input**:
    - A: Ma trận vuông đầu vào
- **Output**: Một đa thức, là định thức của ma trận vuông đầu vào
- **Mục đích**: Hàm tính định thức của ma trận vuông đầu vào
- **Hoạt động**:
    - Dùng đệ quy để tính định thức của ma trận đầu vào
    - Nếu ma trận đầu vào là ma trận 1x1 thì trả về phần tử duy nhất của ma trận
    - Nếu ma trận đầu vào là ma trận 2x2 thì trả về tích của hai đường chéo chính trừ tích của hai đường chéo phụ
    - Nếu ma trận đầu vào là ma trận vuông có kích thước lớn hơn 2x2 thì duyệt qua từng phần tử của hàng đầu tiên, tính định thức của ma trận con bỏ đi hàng và cột chứa phần tử đang xét, nhân với phần tử đang xét, lần lượt cộng dồn các kết quả này lại để tạo thành đa thức định thức của ma trận đầu vào


## Gauss_elimination

- **Input**:
    - matrix: Ma trận mở rộng của hệ phương trình
- **Output**: Ma trận có dạng bậc thang có được từ ma trận mở rộng đầu vào
- **Mục đích**: Dùng phép khử Gauss để biến đổi ma trận mở rộng ban đầu của hệ phương trình về dạng bậc thang
- **Hoạt động**:
    - Khởi tạo pivot là phần tử đầu tiên của matrix
    - *Bước 1*. Kiểm tra nếu phần tử pivot = 0 thì tìm các dòng dưới có phần tử cùng cột với pivot khác 0 và đổi chỗ hai dòng
        - Nếu cả dòng bằng 0 thì chuyển sang cột kế tiếp và thực hiện lại bước 1
    - *Bước 2*. Nhân dòng chứa pivot với $\frac{1}{pivot}$ để có số dẫn đầu là 1
    - *Bước 3*. Cộng một bội số thích hợp của dòng đầu cho từng dòng dưới để biến các số hạng bên dưới thành 0
        - Vì pivot là 1 nên bội số chính là các số hạng của dòng dưới
    - *Bước 4*. Chuyển sang pivot của cột kế tiếp và lặp lại bước 1 đến khi đạt được ma trận bậc thang


## back_substitution

- **Input**:
    - matrix: Ma trận có dạng bậc thang từ ma trận mở rộng của hệ phương trình
- **Output**: Nghiệm của hệ phương trình (trường hợp nghiệm duy nhất/ vô số nghiệm) hoặc thông báo hệ phương trình vô nghiệm.
- **Mục đích**: Thế ngược ma trận bậc thang để tìm nghiệm của hệ phương trình
- **Hoạt động**:
    - Khai báo
        - solutions: Danh sách chứa nghiệm
    - **Bước 1**. Xét số dòng toàn 0 của ma trận hệ số
        - Nếu hệ số tự do tương ứng của dòng toàn 0 khác 0 => Hệ phương trình **vô nghiệm**
        - Nếu hệ số tự do tương ứng của dòng toàn 0 bằng 0 => Đếm số dòng toàn 0 lưu vào biến zero_row
    - **Bước 2**. Tính số dòng khác không non_zero_row = m - zero_row
        - Nếu non_zero_row = số ẩn (n-1) => **Nghiệm duy nhất** => Thế ngược từng dòng để giải từ dòng dưới cùng lên trên
        - Nếu non_zero_row < số ẩn (n-1) => **Vô số nghiệm**
            
            - Ý tưởng giải trường hợp vô số nghiệm: 
                + Dùng 1 list variables 2 chiều để lưu hệ số của từng ẩn ứng với từng biến tùy ý
                + Dùng 1 list solutions 1 chiều để lưu hệ số tự do của ẩn
                + variables là list chứa các vector riêng sau khi giải hệ phương trình
                
                Ví dụ: x2= 1 + 2α -3β
                + 1 là hệ số tự do
                + 2 và -3 là hệ số của ẩn x2 với biến tùy ý α và β
            - Ví dụ:
                Hệ phương trình            
                    $$
                \left(\begin{array}{cccc|c}
                4 & -2 & -4 & 2 & 1 \\
                6 & -3 & 0 & -5 & 3 \\
                8 & -4 & 28 & -44 & 11 \\
                -8 & 4 & -4 & 12 & -5 \\
                \end{array}\right)\rightarrow
                \left(\begin{array}{cccc|c}
                1 & -0.5 & -1 & 0.5 & 0.25 \\
                0 & 0 & 1 & -1.33 & 0.25 \\
                0 & 0 & 0 & 0 & 0 \\
                0 & 0 & 0 & 0 & 0 \\
                \end{array}\right)
                $$              
                
                 Khởi tạo ban đầu:
                $variables =\begin{bmatrix}
                    0 & 1 & 0 & 0\\
                    0 & 0 & 0 & 1
                \end{bmatrix}$
                
                Giải thích
                - Số dòng của variables là số biến tùy ý:
                    + variable[0] chứa các hệ số alpha của các ẩn
                    + variable[1] chứa các hệ số beta của các ẩn
                    + ...
                - Mỗi dòng của variables chứa các hệ số của từng ẩn đối với biến tùy ý                
                    Ở ví dụ này có 2 biến tùy ý                    
                    + x2 = α <=> x2 = 1α + 0β => variables[0][1]=1 và variables [1][1] = 0
                    + x4 = β <=> x4 = 0α + 1β => variables[1][3]=1 và variables [0][3] = 0
                
                Nghiệm của hệ phương trình: 
                $\begin{bmatrix}0.50 + 0.50α + 0.83β\\ 0.00 + 1.00α + 0.00β\\ 0.25 + 0.00α + 1.33β\\ 0.00 + 0.00α + 1.00β\end{bmatrix}$
                
                Khi đó:
                $variables =\begin{bmatrix}
                    0.5 & 1 & 0 & 0\\
                    0.83 & 0 & 1.33 & 1
                \end{bmatrix}            
                solutions =\begin{bmatrix}
                    0.5 & 0 & 0.25 & 0
                \end{bmatrix}$
                Lúc này variables chứa 2 vector riêng
                **Thực hiện giải trường hợp vô số nghiệm**
                - Bước 1.
                    + Tính số nghiệm tùy ý: count = n - 1 - non_zero_row
                    + Khai báo variables
                - Bước 2. Duyệt pivot từng dòng, đánh dấu các ẩn có biến tùy ý vào temp_list
                    + Như ma trận bậc thang ở ví dụ trên có hai dòng, có pivot ở vị trí index là 1 và 3
                    + => temp_list[0] và temp_list[2] bằng 0 (vì thứ tự mảng bắt đầu từ 0)
                    + => x2 và x4 là 2 ẩn có biến tùy ý => temp_list[1] và temp_list[3] bằng 1 (1 tức là có biến tùy ý)                   
                    + => temp_list = [0,1,0,1]
                - Bước 3. Thực hiện khởi tạo giá trị cho variables bởi các ẩn có biến tùy ý (Phần khởi tạo ban đầu ở ví dụ phần ý tưởng)
                - Bước 4.
                    + Duyệt từng dòng từ dưới lên
                    + Thế ngược để giải hệ phương trình
    
        




## inverse

- **Input**:
    - matrix: Ma trận vuông đầu vào
- **Output**: Ma trận nghịch đảo từ ma trận vuông đầu vào hoặc thông báo ma trận không khả nghịch
- **Mục đích**: Tìm ma trận nghịch đảo của ma trận vuông đầu vào
- **Hoạt động**:
    - Ghép ma trận đơn vị vào ma trận vuông đầu vào để tạo thành ma trận mở rộng
    - Biến đổi Gauss-Jordan
        + Xác định phần tử pivot: Với mỗi cột, xác định phần tử pivot. Nếu phần tử pivot bằng 0, tìm dòng khác có phần tử khác 0 để hoán vị hai dòng. Nếu không tìm được dòng nào khác không thì kết luận ma trận không khả nghịch và kết thúc.
        + Chia cả dòng cho phần tử pivot: Chia dòng cho giá trị của pivot để đưa pivot về giá trị 1.
        + Biến đổi các dòng còn lại: Thực hiện biến đổi để đưa các số hạng trên cùng cột của các dòng khác về 0.
    - Trả về ma trận nghịch đảo bằng cách lấy các cột bên phải của ma trận sau khi đã hoàn thành các biến đổi.
    
## diagonalize

- **Input**:
    - A: Ma trận đầu vào có thể chéo hóa được

- **Output**: Ma trận chéo P, P^-1 và ma trận đường chéo D

- **Mục đích**: Tìm ma trận chéo P, P^-1 và ma trận đường chéo D sao cho A = PDP^-1

- **Hoạt động**:
    - Tính đa thức $det(A - λI)$ nhờ vào thư viện sympy để có được đa thức ẩn $λ$
    - Tìm các giá trị riêng của A bằng cách giải phương trình $det(A - λI) = 0$ bằng hàm solve của sympy
    - Với mỗi giá trị riêng $λ_i$, tìm các vector riêng của A bằng cách giải hệ phương trình $(A - λ_iI)x$ = 0 bằng hàm `Gauss_elimination` và `back_substitution`
    - Nếu số lượng vector riêng tìm được bằng số chiều của ma trận đầu vào thì ma trận đầu vào có thể chéo hóa được
        - Tạo ma trận $P$ bằng cách ghép các vector riêng tìm được theo thứ tự
        - Tính ma trận $P^-1$ là  ma trận nghịch đảo của ma trận $P$ bằng hàm `inverse`
        - Tính ma trận đường chéo $D$ theo công thức $D = P^-1AP$ dùng hàm `dot_matrix`
        - Trả về ma trận $P$, $P^-1$ và $D$
    - Nếu số lượng vector riêng tìm được nhỏ hơn số chiều của ma trận đầu vào thì ma trận đầu vào không thể chéo hóa được


# Mở rộng hàm/ phương thức của các thư viện khác


### Thư viện NumPy

In [5]:
import numpy as np

# Ma trận A (ví dụ)
A = np.array([[-7,0,5],
              [0, 3, 0],
              [-10, 0, 8]])

# Tìm eigenvalues và eigenvectors của A
eigenvalues, eigenvectors = np.linalg.eig(A)

# Ma trận chéo P
P = eigenvectors

# Ma trận đường chéo D
D = np.diag(eigenvalues)

# Ma trận chéo nghịch đảo P^-1
P_inv = np.linalg.inv(P)
print(eigenvalues)

print("Ma trận chéo P:")
print(P)

print("Ma trận chéo P^-1:")
print(P_inv)

print("Ma trận đường chéo D:")
print(D)

[-2.  3.  3.]
Ma trận chéo P:
[[-0.70710678 -0.4472136   0.        ]
 [ 0.          0.          1.        ]
 [-0.70710678 -0.89442719  0.        ]]
Ma trận chéo P^-1:
[[-2.82842712 -0.          1.41421356]
 [ 2.23606798 -0.         -2.23606798]
 [ 0.          1.          0.        ]]
Ma trận đường chéo D:
[[-2.  0.  0.]
 [ 0.  3.  0.]
 [ 0.  0.  3.]]


### Thư viện SciPy

In [10]:
import numpy as np
import scipy.linalg as la

# Ma trận A (ví dụ)
A = np.array([[2, 1, 0],
              [0, 3, 1],
              [0, 0, 4]])

# Tìm eigenvalues và eigenvectors của A
eigenvalues, eigenvectors = la.eig(A)

# Ma trận chéo P
P = eigenvectors

# Ma trận đường chéo D
D = np.diag(eigenvalues)

# Ma trận chéo nghịch đảo P^-1
P_inv = la.inv(P)

print("Ma trận chéo P:")
print(P)

print("Ma trận chéo P^-1:")
print(P_inv)

print("Ma trận đường chéo D:")
print(D)


Ma trận chéo P:
[[1.         0.70710678 0.33333333]
 [0.         0.70710678 0.66666667]
 [0.         0.         0.66666667]]
Ma trận chéo P^-1:
[[ 1.         -1.          0.5       ]
 [ 0.          1.41421356 -1.41421356]
 [ 0.          0.          1.5       ]]
Ma trận đường chéo D:
[[2.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 3.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 4.+0.j]]


### Thư viện SymPy

In [11]:
import sympy as sp

# Ma trận A (ví dụ)
A = [[2, 1, 0],
               [0, 3, 1],
               [0, 0, 4]]
size = len(A)

A = sp.Matrix(A)
# Tìm eigenvalues và eigenvectors của A
eigenvecs = A.eigenvects()

# Ma trận chéo P
P = sp.Matrix([eigenvec for _, _, eigenvec in eigenvecs]).T
# Reshape lại ma trận P
P = P.reshape(size, size)
# Ma trận đường chéo D
D = sp.diag(*[eigenval for eigenval, _, _ in eigenvecs])

# Ma trận chéo nghịch đảo P^-1
P_inv = P.inv()

print("Ma trận chéo P:")
print(P)

print("Ma trận chéo P^-1:")
print(P_inv)

print("Ma trận đường chéo D:")
print(D)




Ma trận chéo P:
Matrix([[1, 0, 0], [1, 1, 0], [1/2, 1, 1]])
Ma trận chéo P^-1:
Matrix([[1, 0, 0], [-1, 1, 0], [1/2, -1, 1]])
Ma trận đường chéo D:
Matrix([[2, 0, 0], [0, 3, 0], [0, 0, 4]])


## So sánh kết quả

Vậy kết quả của ba thư viện và thuật toán ở trên đều ra kết quả giống nhau.

# Ứng dụng của chéo hóa ma trận

- Tính toán ma trận mũ: Khi ma trận đã được chéo hóa, việc tính toán ma trận mũ (nghĩa là ma trận nhân chính nó lặp đi lặp lại) trở nên dễ dàng hơn rất nhiều. Điều này hữu ích trong nhiều lĩnh vực, bao gồm xử lý tín hiệu, kiểm soát hệ thống và thống kê.

- Giải hệ phương trình vi phân: Chéo hóa ma trận giúp chúng ta dễ dàng giải các hệ phương trình vi phân thông qua việc tính toán các ma trận mũ.

- Xử lý ảnh và xử lý tín hiệu: Trong xử lý ảnh và tín hiệu số, chéo hóa ma trận giúp phân tích và trích xuất thông tin quan trọng từ dữ liệu. Chéo hóa ma trận có thể được sử dụng để lọc nhiễu, khôi phục hình ảnh và tín hiệu, và giảm nhiễu

- Giảm chiều dữ liệu: Chéo hóa ma trận có thể được sử dụng để giảm chiều dữ liệu bằng cách loại bỏ các giá trị không cần thiết, từ đó giúp tăng tốc quá trình xử lý và giảm tải tính toán.

- Ứng dụng trong lý thuyết đồ thị: Khi biểu diễn các mạng phức tạp bằng ma trận, việc chéo hóa ma trận giúp chúng ta hiểu rõ cấu trúc và tính chất của mạng.

- Trong khoa học máy tính, chéo hóa ma trận có thể được sử dụng để tính giá trị riêng và vectơ riêng của ma trận, được sử dụng trong nhiều thuật toán khác nhau, chẳng hạn như phân tích chính yếu và phân tích thành phần chính.

- Trực quan hóa dữ liệu: Chéo hóa ma trận giúp trực quan hóa dữ liệu phức tạp bằng cách biểu diễn chúng dưới dạng các phần tử đường chéo, làm cho việc hiểu thông tin trở nên dễ dàng hơn.

- Phân tích và dự đoán chuỗi thời gian: Chéo hóa ma trận có thể được sử dụng để phân tích và dự đoán chuỗi thời gian, ví dụ như trong tài chính và kinh tế (giá cổ phiếu, lãi suất và tỷ giá hối đoái)

- Kiểm tra tính đối xứng và định dạng chuẩn: Chéo hóa ma trận giúp kiểm tra tính đối xứng và định dạng chuẩn của ma trận, từ đó xác định tính chất cơ bản của ma trận và ứng dụng trong các lĩnh vực khác nhau.

- Trong kỹ thuật, chéo hóa ma trận có thể được sử dụng để giải hệ phương trình vi phân, mô phỏng động học của hệ thống, và tối ưu hóa thiết kế.